## Multirun experiment pipeline (WheatFspm)

The following notebook establishes a generalized pipeline for evaluating a computing reservoir against a given task, given multiple experimental runs of the same reservoir.


In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
sys.path.insert(1, os.path.join(sys.path[0], '../../'))  # for importing local packages from src

In [2]:
DATASET_NEMA_H0 = '../datasets/dataset_NEMA_NEMA_H0.csv'
DATASET_NEMA_H3 = '../datasets/dataset_NEMA_NEMA_H3.csv'
DATASET_NEMA_H15 = '../datasets/dataset_NEMA_NEMA_H15.csv'

### Loading the datasets

These datasets were collected and converted in the WheatFspm repository.

There are three simulations made available in the WheatFspm repository that are useable for RC experiments: NEMA H0, H3 and H15.

We can try using these datasets in two different ways:

1. Treat every dataset as a separate plant, training a readout for each simulation run.
2. Concatenating the three datasets as observed behavior of a single plant.

In [3]:
from src.model.rc_dataset import ExperimentDataset

dataset_nema_h0 = ExperimentDataset(csv_path=DATASET_NEMA_H0)
dataset_nema_h3 = ExperimentDataset(csv_path=DATASET_NEMA_H3)
dataset_nema_h15 = ExperimentDataset(csv_path=DATASET_NEMA_H15)

datasets = [
  ('NEMA_H0', dataset_nema_h0), 
  ('NEMA_H3', dataset_nema_h3), 
  ('NEMA_H15', dataset_nema_h15)
]

### Defining targets and observed state variables

These were selected in a previous notebook, `2022_03_23_wheatfspm_dataset_inspection.ipynb` and are defined in a config file for reuse among notebooks.

In [4]:
%reload_ext autoreload
%autoreload 2 

from model_config import targets, state_variables

print(f'Targets:')
for target in targets:
  print(f'\t- {target}')

print(f'\nState variables:')
for state_var in state_variables:
  print(f'\t- {state_var}')

Targets:
	- input_air_temperature
	- input_humidity
	- input_PARi
	- output__axes__Total_Transpiration
	- output__axes__C_exudated
	- output__axes__SAM_temperature
	- output__axes__delta_teq
	- output__axes__sum_respi_shoot
	- output__organ_roots__N_exudation

State variables:
	- state__An
	- state__Transpiration
	- state__S_Sucrose
	- state__Ts
	- state__gs
	- state__Ag
	- state__Tr
	- state__sucrose
	- state__Rd
	- state__sum_respi
	- state__Photosynthesis
	- state__PARa


### Data preprocessing, grouping and train-test splitting

The available datasets will be processed into 4 datasets:

- NEMA_H0
- NEMA_H3
- NEMA_H15
- NEMA_COMBINED (concatenated as data from the same plant)

In [5]:
from src.learning.preprocessing import generate_mask


WARMUP_STEPS = 4 * 24
DAY_MASK = generate_mask(5, 21)

In [6]:
from wheatfspm_pipeline_utils import preprocess_data, group_by_day, train_test_split_alternating
from wheatfspm_pipeline_utils import direct_target_generator, direct_reservoir_generator
from wheatfspm_pipeline_utils import preprocess_raw_X
from model_config import max_time_step

def generate_X_y_groups(datasets, target, state_var, target_generator, state_generator):
  """Generates X, y and groups arrays for each dataset, plus a concatenated dataset.
     NOTE: The groups in the concatenated dataset are such that the same calendar day is in the same group.

     Also generates a baseline dataset where the reservoir is just a combination of all environmental inputs.
  """
  data = {}

  # Preprocess the data for each dataset
  for name, dataset in datasets:
    target_data = next(target_generator(dataset, target, name))
    reservoir_data = next(state_generator(dataset, state_var, name))
    X_raw, y_raw = preprocess_data(dataset, target_data, reservoir_data, WARMUP_STEPS, DAY_MASK)
    X, y = X_raw[0, :, :], y_raw[0, :]
    groups = group_by_day(X, DAY_MASK)
    data[name] = (X, y, groups)

  # Generate the concatenated dataset
  all_arrays = list(data.values())
  X_combined = np.concatenate(list(map(lambda x : x[0], all_arrays)))
  y_combined = np.concatenate(list(map(lambda x : x[1], all_arrays)))
  groups_combined = np.concatenate(list(map(lambda x : x[2], all_arrays)))
  data['combined'] = (X_combined, y_combined, groups_combined)

  # Generate the baseline dataset
  # baseline_name, baseline_dataset = datasets[0]
  # input_vars = baseline_dataset.get_input_variables()
  # X_baseline = np.empty((max_time_step[baseline_name], len(input_vars)))
  # for i, var in enumerate(input_vars):
  #   X_baseline[:, i] = baseline_dataset.get_target(var, baseline_name)[:max_time_step[baseline_name]]
  # X_baseline = preprocess_raw_X(X_baseline, WARMUP_STEPS, DAY_MASK)
  # y_baseline = data[baseline_name][1]
  # groups_baseline = data[baseline_name][2]
  # data[f'baseline_{baseline_name}'] = X_baseline, y_baseline, groups_baseline

  return data

In [7]:
TARGET = 'output__axes__C_exudated' # targets[0]
STATE_VAR = 'state__Ts' # state_variables[0]

preprocessed_data = generate_X_y_groups(datasets, TARGET, STATE_VAR, direct_target_generator, direct_reservoir_generator)

for name, (X, y, groups) in preprocessed_data.items():
  print(f'{name}:')
  print(f'\tX: {X.shape}')
  print(f'\ty: {y.shape}')
  print(f'\tgroups: {len(np.unique(groups))} (shape {groups.shape})')

NEMA_H0:
	X: (400, 10)
	y: (400,)
	groups: 25 (shape (400,))
NEMA_H3:
	X: (512, 10)
	y: (512,)
	groups: 32 (shape (512,))
NEMA_H15:
	X: (544, 10)
	y: (544,)
	groups: 34 (shape (544,))
combined:
	X: (1456, 10)
	y: (1456,)
	groups: 34 (shape (1456,))


### Model definition

- Readout model is a standard RidgeRegression model with intercept term and CV-tuned regularization strength $\alpha$.
- CV search grid is a progression of logarithmicly spaced values for regularization strength $\alpha$.
- CV and testing metric is NMSE.

In [8]:
N_FOLDS = 3;

In [9]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupKFold

from src.learning.scorers import nmse_scorer

# Define model
readout = Pipeline([
  ('ridge_regression', Ridge(alpha=1, fit_intercept=True))
])

# define search grid
search_grid = [{
  'ridge_regression__alpha': 10 ** np.linspace(np.log10(1e-4), np.log10(1e2), 50)
}]

# define cross-validation and testing metric
scorer = nmse_scorer

# Define CV fold strategy
folds = GroupKFold(n_splits=N_FOLDS)

### Generating a manifest of all experiments to run

Currently we are only benchmarking direct target prediction, but in the future there will be other tasks generated from the base targets as well. These will be generated in this section.


In [10]:
TARGETS = [(target_name, direct_target_generator) for target_name in targets]
STATE_VARS = [(state_var, direct_reservoir_generator) for state_var in state_variables]

TRAIN_TEST_RATIO = 1

### Fitting all readout functions

Process:

- For each target:
  - For each observed state variable:
    - For each dataset:
      1. Preprocess the data
      2. Fit for each dataset
      3. Store the resulting training, cross-validation and test scores.

In [11]:
from tqdm import tqdm
from src.learning.training import perform_gridsearch


total_loops = len(targets) * len(state_variables) * (len(preprocessed_data))
print(f'Performing {total_loops} fits...')


models = {}
results = []


with tqdm(total=total_loops) as pbar:

    for target_name, target_generator in TARGETS:
        for state_var, state_generator in STATE_VARS:

            # Preprocess data for model fit
            preprocessed_data = generate_X_y_groups(datasets, target_name, state_var, target_generator, state_generator)

            # For each dataset combination
            for dataset_name, (X, y, groups) in preprocessed_data.items():
                train, test = train_test_split_alternating(X, y, groups, ratio=TRAIN_TEST_RATIO)

                # fit model
                X_train, y_train, groups_train = train
                model, scores = perform_gridsearch(readout, X_train, y_train, groups_train, folds, search_grid, verbose=False)
                (train_mean, train_std), (cv_mean, cv_std) = scores

                # Determine test score
                X_test, y_test, _ = test
                test_score = scorer(model, X_test, y_test)

                # store results
                models[(target_name, state_var, dataset_name)] = model
                results.append({
                    'target': target_name,
                    'state_var': state_var,
                    'dataset': dataset_name,
                    'test_score': test_score,
                    'train_mean': train_mean,
                    'train_std': train_std,
                    'cv_mean': cv_mean,
                    'cv_std': cv_std
                })

                pbar.update(1)                

Performing 432 fits...


100%|██████████| 432/432 [00:46<00:00,  9.35it/s]


### Storing experiment results

Storing all the fit results into a Pandas dataframe, then storing it as a CSV file for further inspection in another notebook.

In [12]:
results_df = pd.DataFrame.from_dict(results)
results_df.set_index(['target', 'state_var', 'dataset'])
results_df.to_csv(f'test_results_nonan.csv')
results_df.head(10)

,target,state_var,dataset,test_score,train_mean,train_std,cv_mean,cv_std
0,input_air_temperature,state__An,NEMA_H0,-0.399352,-0.230236,0.019201,-0.394548,0.081092
1,input_air_temperature,state__An,NEMA_H3,-0.652902,-0.195643,0.024072,-0.279485,0.062588
2,input_air_temperature,state__An,NEMA_H15,-0.393598,-0.306023,0.013792,-0.473073,0.063097
3,input_air_temperature,state__An,combined,-0.373257,-0.263725,0.018267,-0.375859,0.041791
4,input_air_temperature,state__Transpiration,NEMA_H0,-0.469686,-0.200324,0.024970,-0.556837,0.137555
5,input_air_temperature,state__Transpiration,NEMA_H3,-0.458527,-0.299510,0.014847,-0.454499,0.029938
6,input_air_temperature,state__Transpiration,NEMA_H15,-0.437393,-0.289362,0.043631,-0.454884,0.159976
7,input_air_temperature,state__Transpiration,combined,-0.424136,-0.283927,0.048285,-0.449779,0.077963
8,input_air_temperature,state__S_Sucrose,NEMA_H0,-1.078103,-0.442371,0.025044,-0.789191,0.151156
9,input_air_temperature,state__S_Sucrose,NEMA_H3,-0.926533,-0.524523,0.037205,-0.766124,0.040170


In [13]:
results_sorted = results_df.sort_values('test_score', ascending=False)
results_sorted.head(10)

,target,state_var,dataset,test_score,train_mean,train_std,cv_mean,cv_std
148,output__axes__Total_Transpiration,state__Transpiration,NEMA_H0,-0.000001,-0.000001,5.585133e-08,-0.000004,0.000002
149,output__axes__Total_Transpiration,state__Transpiration,NEMA_H3,-0.000003,-0.000002,2.579223e-07,-0.000010,0.000006
150,output__axes__Total_Transpiration,state__Transpiration,NEMA_H15,-0.000004,-0.000007,2.701657e-06,-0.000027,0.000014
151,output__axes__Total_Transpiration,state__Transpiration,combined,-0.000112,-0.000108,9.053750e-06,-0.000128,0.000011
252,output__axes__SAM_temperature,state__Ts,NEMA_H0,-0.000814,-0.000606,1.243477e-04,-0.001672,0.000562
168,output__axes__Total_Transpiration,state__Tr,NEMA_H0,-0.013107,-0.007619,1.321665e-03,-0.009096,0.002893
301,output__axes__delta_teq,state__Ts,NEMA_H3,-0.017909,-0.043492,1.543245e-02,-0.059045,0.031995
303,output__axes__delta_teq,state__Ts,combined,-0.018560,-0.034040,1.335203e-02,-0.043242,0.024341
302,output__axes__delta_teq,state__Ts,NEMA_H15,-0.018675,-0.039571,1.178157e-02,-0.049990,0.018475
300,output__axes__delta_teq,state__Ts,NEMA_H0,-0.019942,-0.014285,2.465289e-03,-0.022020,0.007200


In [14]:
results_sorted_top = results_sorted[results_sorted['test_score'] > -0.5]
results_sorted_top.groupby('target').count()

,state_var,dataset,test_score,train_mean,train_std,cv_mean,cv_std
target,,,,,,,
input_PARi,46,46,46,46,46,46,46
input_air_temperature,20,20,20,20,20,20,20
input_humidity,7,7,7,7,7,7,7
output__axes__C_exudated,30,30,30,30,30,30,30
output__axes__SAM_temperature,20,20,20,20,20,20,20
output__axes__Total_Transpiration,44,44,44,44,44,44,44
output__axes__delta_teq,20,20,20,20,20,20,20
output__axes__sum_respi_shoot,1,1,1,1,1,1,1
output__organ_roots__N_exudation,18,18,18,18,18,18,18
